In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 22.3MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2040037523996810719
xla_global_id: -1
]


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [10, 11]
BiLSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in BiLSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(BiLSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))


                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'BiLSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'BiLSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_11",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 13,
  "patience_list": 3,
  "MAE_normalize": 0.040563322564065585,
  "MSE_normalize": 0.0017035428874811129,
  "RMSE_normalize": 0.041273997716251246,
  "MAPE_normalize": 0.04226167622433953,
  "MAE": 1658.919642857143,
  "MSE": 2849290.5992257255,
  "RMSE": 1687.9841821609957,
  "MAPE": 0.021056181591826557
}
Percent CPU Usage: 27.1
Percent Ram Usage: 10.3
2022-10-12 10:37:15.158891+07:00
PROGRESS: [ 2 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_11",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 11,
  "patience_list": 3,
  "MAE_normalize": 0.09066426217849778,
  "MSE_normalize": 0.008378906026638214,
  "RMSE_normalize": 0.09153636450415875,
  "MAPE_normalize": 0.09457164754356621,
  "MAE":

{
  "model": "BiLSTM",
  "AHB_number": "AHB_11",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 17,
  "patience_list": 3,
  "MAE_normalize": 0.14258426743188088,
  "MSE_normalize": 0.020529993710682145,
  "RMSE_normalize": 0.14328291492945747,
  "MAPE_normalize": 0.14889761970821608,
  "MAE": 5831.265625,
  "MSE": 34337704.4175851,
  "RMSE": 5859.8382586539965,
  "MAPE": 0.07409964218989053
}
Percent CPU Usage: 56.7
Percent Ram Usage: 11.0
2022-10-12 10:37:41.031278+07:00
PROGRESS: [ 6 / 5040 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 4679.921875,
  "MSE": 22493140.72501046,
  "RMSE": 4742.693404070144,
  "MAPE": 0.0499524908229015
}
Percent CPU Usage: 60.8
Percent Ram Usage: 16.2
2022-10-13 03:53:21.595516+07:00
PROGRESS: [ 4833 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_12",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 26,
  "patience_list": 10,
  "MAE_normalize": 0.07507339473329219,
  "MSE_normalize": 0.0056869638942951325,
  "RMSE_normalize": 0.07541196121501637,
  "MAPE_normalize": 0.07884930613508441,
  "MAE": 3227.628348214286,
  "MSE": 10511759.618905203,
  "RMSE": 3242.1843900224435,
  "MAPE": 0.03448736479008136
}
Percent CPU Usage: 58.7
Percent Ram Usage: 16.2
2022-10-13 03:53:30.458047+07:00
PROGRESS: [ 4834 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_12",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.5,

In [ ]:
name = "all_performance_BiLSTM_5nsteps_AHB11-12"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_BiLSTM_5nsteps_AHB11-12.json /content/drive/MyDrive/Depression
! cp /content/all_performance_BiLSTM_5nsteps_AHB11-12.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,BiLSTM_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,BiLSTM,AHB_11,uni,1,8,0.2,2,5,13,3,0.040563,0.001704,0.041274,0.042262,1658.919643,2.849291e+06,1687.984182,0.021056
1,BiLSTM,AHB_11,mul,1,8,0.2,2,5,11,3,0.090664,0.008379,0.091536,0.094572,3707.893973,1.401424e+07,3743.560417,0.047091
2,BiLSTM,AHB_11,uni,1,8,0.2,4,5,21,3,0.043539,0.001948,0.044140,0.045390,1780.598214,3.258659e+06,1805.175637,0.022608
3,BiLSTM,AHB_11,mul,1,8,0.2,4,5,12,3,0.014769,0.000314,0.017724,0.015180,604.000000,5.254100e+05,724.851742,0.007615
4,BiLSTM,AHB_11,uni,1,8,0.2,8,5,17,3,0.142584,0.020530,0.143283,0.148898,5831.265625,3.433770e+07,5859.838259,0.074100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,BiLSTM,AHB_12,mul,3,128,0.5,32,5,40,10,0.071195,0.005157,0.071809,0.074681,3060.866071,9.531309e+06,3087.281875,0.032687
5036,BiLSTM,AHB_12,uni,3,128,0.5,64,5,59,10,0.024076,0.000606,0.024617,0.025199,1035.112723,1.120168e+06,1058.379828,0.011043
5037,BiLSTM,AHB_12,mul,3,128,0.5,64,5,37,10,0.085219,0.007346,0.085711,0.089464,3663.832589,1.357894e+07,3684.961429,0.039140
5038,BiLSTM,AHB_12,uni,3,128,0.5,128,5,67,10,0.004373,0.000032,0.005660,0.004536,188.016741,5.921800e+04,243.347494,0.001998
